In [ ]:
%load_ext autoreload
%autoreload 2

from context import uncertify

In [ ]:
import logging
from uncertify.log import setup_logging
setup_logging()
LOG = logging.getLogger(__name__)

# Matplotlib DEBUG logging spits out a whole bunch of crap
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
import itertools

import torch
import torchvision
import seaborn as sns

from uncertify.models.vae import VariationalAutoEncoder
from uncertify.models.vae_baur2020 import BaurDecoder, BaurEncoder
from uncertify.data.dataloaders import dataloader_factory, DatasetType
from uncertify.visualization.reconstruction import plot_stacked_scan_reconstruction_batches
from uncertify.deploy import yield_reconstructed_batches
from uncertify.visualization.grid import imshow_grid
from uncertify.common import DATA_DIR_PATH

In [ ]:
model = VariationalAutoEncoder(BaurEncoder(), BaurDecoder(), get_batch_fn=lambda batch: batch['scan'])

In [ ]:
CHECKPOINT_PATH = DATA_DIR_PATH / 'lightning_logs/train_vae/version_32/checkpoints/epoch=2.ckpt'
assert CHECKPOINT_PATH.exists(), f'Model checkpoint does not exist!'

checkpoint = torch.load(CHECKPOINT_PATH)
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
_, brats_val_dataloader = dataloader_factory(DatasetType.BRATS17, batch_size=4, shuffle_val=True)
camcan_train_dataloader, camcan_val_dataloader = dataloader_factory(DatasetType.CAMCAN, batch_size=4)

In [ ]:
plot_n_batches = 3

batch_generator = yield_reconstructed_batches(brats_val_dataloader, model)
plot_stacked_scan_reconstruction_batches(batch_generator, plot_n_batches, cmap='hot', axis='off', figsize=(10, 10))